<a href="https://colab.research.google.com/github/getdevelop/Praveen_Basavaraj_/blob/main/Subsidary_Idea%3D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Florence-2-large sample usage

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.10

In [ ]:
# pip install timm flash_attn

In [ ]:
!pip install splacy
# splacy it NLP library used to check the simlarity between 2 text meaning
!python3 -m spacy download en_core_web_lg

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy
import torch
import splacy
%matplotlib inline

In [1]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

nlp = spacy.load("en_core_web_lg")

In [ ]:
# url = "" provide your image link
# image = Image.open(requests.get(url, stream=True).raw)
image=Image.open("/content/images.jpeg")



## define the prediction function

In [ ]:
def run_example(task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer


In [ ]:
# input the 2 image path to genrate the capttion

caption1 = run_example(task_prompt, image1)
caption2 = run_example(task_prompt, image2)


###  Caption are Generated from the given image

In [5]:
task_prompt = '<CAPTION>'
run_example(task_prompt)

In [4]:
task_prompt = '<DETAILED_CAPTION>'
run_example(task_prompt)

In [3]:
task_prompt = '<DETAILED_CAPTION>'
run_example(task_prompt)

In [2]:
task_prompt = '<MORE_DETAILED_CAPTION>'
run_example(task_prompt)

In [ ]:
# import spacy

# nlp = spacy.load("en_core_web_lg")

# s1 = nlp("The image shows a dairy barn with a few cows in it, surrounded by water at the bottom and a shed in the background. The sky is visible at the top of the image, and there are a few buildings and trees in the distance.")
# s2 = nlp("The image shows a large group of cows in a barn, eating hay from a trough. The cows are black and white in color, and the barn is surrounded by poles and a shed. In the background, there are trees and a clear blue sky.")

# res = s1.similarity(s2)

# if res >= 0.98:
#     print("Both images are similar")
# else:
#     print("Images are not similar")


In [ ]:
def compare_text_similarity(text1, text2):
    s1 = nlp(text1)
    s2 = nlp(text2)
    similarity = s1.similarity(s2)
    if similarity >= 0.98:
        print("Both images are similar")
    else:
        print("Images are not similar")
    return similarity

In [ ]:
similarity_score = compare_text_similarity(caption1, caption2)
print(similarity_score )